In [1]:
import requests
import json
import pandas as pd
from db_connect import EngineConnect as DatabaseConnect
import config
import pickle

In [2]:
class DimUserModel:
    def __init__(
        self,
        *kwargs,
        staff_code: str,
        user_name: str,
        full_name: int,
        job_title: int,
        location: str,
        department: int,
        manager_full_name: str,
        manager_user_name: str,
        work_shift: str
    ):
        self.staff_code = staff_code
        self.user_name = user_name
        self.full_name = full_name
        self.job_title = job_title
        self.location = location
        self.department = department
        self.manager_full_name = manager_full_name
        self.manager_user_name = manager_user_name
        self.work_shift = work_shift

In [3]:
class DimUserExecutor:
    def __init__(
        self,
        *kwargs,
        environment: str,
        url: str,
        schema: str,
        table: str,
        index: list = None,
        db: DatabaseConnect
    ):
        self.environment = environment
        self.url = url
        self.schema = schema
        self.table = table
        self.index = index
        self.db = db
        
    def get_data(self):
        if self.environment == 'development':
            data_load = pickle.load(open('dim_user.pickle', 'rb'))
        else:
            response = requests.get(self.url)
            datatext = response.text
            data_load = json.loads(datatext)
        return data_load
        
    def transform_data(self):
        data = self.get_data()
        col_rename = {
            'StaffCode': 'staff_code', 
            'Username': 'user_name',
            'Fullname': 'full_name',
            'JobTitle': 'job_title',
            'ManagerUserName': 'manager_user_name',
            'ManagerFullName': 'manager_full_name',
            'Department': 'department',
            'Location': 'location',
            'WorkShift': 'work_shift',
        }
        df = pd.json_normalize(data)
        df = df.rename(columns = col_rename)
        if self.index == None:
            self.db.update_df(df, self.schema, self.table)
        else:
            self.db.update_df_with_index(df, self.schema, self.table, self.index)

In [4]:
db_connect = DatabaseConnect(uri = config.DWH_SQLALCHEMY_URI)
dim_user = DimUserExecutor(
    environment = config.ENVIRONMENT,
    url = config.EHR_URL_USER_INFOR,
    schema = config.DWH_ANALYTIC_SCHEMA,
    table = config.DWH_DIM_USER_TABLE,
    db = db_connect,
    index = ['user_name']
)
dim_user.transform_data()